1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

In [1]:
import numpy as np
import pandas as pd
import warnings
import requests
from io import StringIO
from kmodes.kmodes import KModes
from scipy.special import logsumexp
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI, fowlkes_mallows_score as FMI
from sklearn.preprocessing import OneHotEncoder

In [2]:
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 300)

In [4]:
dataset_urls = {
    "Soybean (Small)": "https://archive.ics.uci.edu/ml/machine-learning-databases/soybean/soybean-small.data",
    "Zoo": "https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data",
    "Heart Disease": "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
    "Breast Cancer Wisconsin (Original)": "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",
    "Dermatology": "https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data",
    "Letter Recognition (E, F)": "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data",
    "Molecular Biology (Splice-junction Gene Sequences)": "https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/splice-junction-gene-sequences/splice.data",
    "Mushroom": "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
}

In [5]:
n_clusters_dict = {
    "Soybean (Small)": 4,
    "Zoo": 7,
    "Heart Disease": 2,
    "Breast Cancer Wisconsin (Original)": 2,
    "Dermatology": 6,
    "Letter Recognition (E, F)": 2,
    "Molecular Biology (Splice-junction Gene Sequences)": 3,
    "Mushroom": 2,
}

In [6]:
num_runs = 10

In [7]:
dataframes = {}  

In [8]:
for name, url in dataset_urls.items():
    response = requests.get(url, verify=False)
    data = response.text
    
    data_io = StringIO(data)
    df = pd.read_csv(data_io, header=None)
    
    df = df.replace('?', np.nan) 
    df = df.dropna()  

       if name == "Letter Recognition (E, F)":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Mushroom":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Molecular Biology (Splice-junction Gene Sequences)":
        y = df.iloc[:, 0].str.strip()
        X = pd.DataFrame([list(seq.strip()) for seq in df.iloc[:, 2]])
    else:
        X, y = df.iloc[:, :-1], df.iloc[:, -1]

    
    for col in X.columns:
        if len(X[col].unique()) <= 1:
            X.drop(columns=[col], inplace=True)
    
    
    dataframes[name] = {'features': X, 'targets': y}

In [9]:
def preprocess_datasets(dataframes):
    if 'Zoo' in dataframes:
        zoo_df = dataframes['Zoo']['features']
        zoo_df = zoo_df.drop(columns=[0])
        dataframes['Zoo']['features'] = zoo_df

    if 'Heart Disease' in dataframes:
        hd_df = dataframes['Heart Disease']['features']
        columns_to_drop = [0, 3, 4, 7, 9]
        hd_df = hd_df.drop(columns=hd_df.columns[columns_to_drop])
        dataframes['Heart Disease']['features'] = hd_df
        y_hd = dataframes['Heart Disease']['targets']
        dataframes['Heart Disease']['targets'] = y_hd.apply(lambda x: 0 if x == 0 else 1)
    
    if 'Breast Cancer Wisconsin (Original)' in dataframes:
        bcw_df = dataframes['Breast Cancer Wisconsin (Original)']['features']
        bcw_df = bcw_df.drop(columns=bcw_df.columns[0])
        dataframes['Breast Cancer Wisconsin (Original)']['features'] = bcw_df
    
    if 'Dermatology' in dataframes:
        derm_df = dataframes['Dermatology']['features']
        derm_df = derm_df.drop(columns=derm_df.columns[-1])
        dataframes['Dermatology']['features'] = derm_df

    if 'Letter Recognition (E, F)' in dataframes:
        lr_ef_df = dataframes['Letter Recognition (E, F)']['features']
        lr_ef_targets = dataframes['Letter Recognition (E, F)']['targets']
        mask = lr_ef_targets.isin(['E', 'F'])
        dataframes['Letter Recognition (E, F)']['features'] = lr_ef_df[mask]
        dataframes['Letter Recognition (E, F)']['targets'] = lr_ef_targets[mask]

    return dataframes

In [10]:
dataframes = preprocess_datasets(dataframes)

In [11]:
class BernoulliMixture:
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def fit(self,x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis

    def get_save_single(self, x, mu):
        x = np.array(x, dtype=np.float64)
        mu = np.array(mu, dtype=np.float64)

        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        try:
            result = np.tensordot(x, np.log(mu_place), axes=(1, 1))
        except TypeError as e:
            print("TypeError encountered:", e)
            print("x shape:", x.shape, "x dtype:", x.dtype)
            print("mu_place shape:", mu_place.shape, "mu_place dtype:", mu_place.dtype)
            raise

        return result

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [12]:
def perform_bernoulli(features, n_components):
    bm = BernoulliMixture(n_components=n_components, max_iter=1000, tol=1e-3)
    bm.fit(features)
    cluster_labels = bm.predict(features)
    return cluster_labels

In [13]:
class StochasticBernoulliMixture:
    def __init__(self, n_components, max_iter, tol=1e-3, n_samples_per_component=10):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.n_samples_per_component = n_samples_per_component  

    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            self.stochastic_e_step()
            self.remember_params()
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if abs(self.logL - self.old_logL) < self.tol:
                break
            if np.isnan(self.logL):
                self.reset_params()
                break

    def stochastic_e_step(self):
        log_probs = np.log(self.pi)[None, :] + self.get_log_bernoullis(self.x)
        log_probs -= logsumexp(log_probs, axis=1)[:, None]
        probs = np.exp(log_probs)
        self.gamma = np.zeros_like(probs)
        for i in range(self.x.shape[0]):
            for _ in range(self.n_samples_per_component):
                sampled_z = np.random.choice(self.n_components, p=probs[i])
                self.gamma[i, sampled_z] += 1
        self.gamma /= self.n_samples_per_component
        
    def get_log_bernoullis(self, x):
        return np.einsum('ij,kj->ik', x, np.log(self.mu)) + np.einsum('ij,kj->ik', 1-x, np.log(1-self.mu))
                
    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis

    def get_save_single(self, x, mu):
        x = np.array(x, dtype=np.float64)
        mu = np.array(mu, dtype=np.float64)

        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        try:
            result = np.tensordot(x, np.log(mu_place), axes=(1, 1))
        except TypeError as e:
            print("TypeError encountered:", e)
            print("x shape:", x.shape, "x dtype:", x.dtype)
            print("mu_place shape:", mu_place.shape, "mu_place dtype:", mu_place.dtype)
            raise

        return result

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self):
        self.mu = np.einsum('ik,id->kd', self.gamma, self.x) / np.sum(self.gamma, axis=0)[:, None]

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [14]:
def perform_stochastic_bernoulli(features, n_components):
    sbm = StochasticBernoulliMixture(n_components=n_components, max_iter=1000, tol=1e-3, n_samples_per_component=10)
    sbm.fit(features)
    cluster_labels = sbm.predict(features)
    return cluster_labels

In [15]:
def perform_kmodes(features, n_clusters):
    """Perform clustering using KModes algorithm."""
    km = KModes(n_clusters=n_clusters, init='random', n_init=5)
    clusters = km.fit_predict(features)
    return clusters

In [16]:
def run_clustering_algorithms(dataframes, n_clusters_dict, num_runs=10):
    results_list = []
    for name, data in dataframes.items():
        print("Processing:", name)
        
        features = data['features']
        true_labels = data['targets'].squeeze()  
        n_clusters = n_clusters_dict.get(name, 2)  
        
        encoder = OneHotEncoder(sparse=False)
        features_encoded = encoder.fit_transform(features)

        metrics = {'Bernoulli Mixture': [], 'Stochastic Bernoulli Mixture': [], 'KModes': []}  
        
        for _ in range(num_runs):
             bm_clusters = perform_bernoulli(features_encoded, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, bm_clusters)
            metrics['Bernoulli Mixture'].append((ari, nmi, fmi))
            
            sbm_clusters = perform_stochastic_bernoulli(features_encoded, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, sbm_clusters)
            metrics['Stochastic Bernoulli Mixture'].append((ari, nmi, fmi))
            
            km_clusters = perform_kmodes(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, km_clusters)
            metrics['KModes'].append((ari, nmi, fmi))

        for method, values in metrics.items():
            ari_vals, nmi_vals, fmi_vals = zip(*values)
            ari_mean, ari_std = np.mean(ari_vals), np.std(ari_vals)
            nmi_mean, nmi_std = np.mean(nmi_vals), np.std(nmi_vals)
            fmi_mean, fmi_std = np.mean(fmi_vals), np.std(fmi_vals)
            results_list.append({
                "Dataset": name,
                "Method": method,
                "ARI": f"{ari_mean:.4f}±{ari_std:.2f}",
                "NMI": f"{nmi_mean:.4f}±{nmi_std:.2f}",
                "FMI": f"{fmi_mean:.4f}±{fmi_std:.2f}"
            })

    results_df = pd.DataFrame(results_list)
    return results_df

In [17]:
def calculate_metrics(true_labels, predicted_labels):
    return ARI(true_labels, predicted_labels), NMI(true_labels, predicted_labels), FMI(true_labels, predicted_labels)

In [18]:
results = run_clustering_algorithms(dataframes, n_clusters_dict, num_runs)

Processing: Soybean (Small)
Processing: Zoo
Processing: Heart Disease
Processing: Breast Cancer Wisconsin (Original)
Processing: Dermatology
Processing: Letter Recognition (E, F)
Processing: Molecular Biology (Splice-junction Gene Sequences)
Processing: Mushroom


## Presentation of Results

In [19]:
def reformat_results(results_df):
    expanded_df = pd.melt(results_df, id_vars=["Dataset", "Method"], value_vars=["ARI", "NMI", "FMI"], var_name="Metric", value_name="Value")
    expanded_df[['Metric_Value', 'Std']] = expanded_df['Value'].str.split('±', expand=True)
    expanded_df.drop(columns=['Value'], inplace=True) 
    
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].astype(float)
    expanded_df['Std'] = expanded_df['Std'].astype(float)

    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].map('{:.4f}'.format) + "±" + expanded_df['Std'].map('{:.2f}'.format)
    
    dataset_order = results_df['Dataset'].unique()
    method_order = results_df['Method'].unique()

    pivot_df = expanded_df.pivot_table(index=["Dataset", "Metric"], columns="Method", values="Metric_Value", aggfunc='first')
    
    pivot_df = pivot_df.reindex(dataset_order, level='Dataset')
    pivot_df = pivot_df.reindex(method_order, axis='columns')

    return pivot_df

In [20]:
formatted_results = reformat_results(results)

In [21]:
print(formatted_results)

Method                                                    Bernoulli Mixture Stochastic Bernoulli Mixture       KModes
Dataset                                            Metric                                                            
Soybean (Small)                                    ARI          1.0000±0.00                  0.9815±0.04  0.9237±0.13
                                                   FMI          1.0000±0.00                  0.9863±0.03  0.9425±0.10
                                                   NMI          1.0000±0.00                  0.9854±0.03  0.9454±0.09
Zoo                                                ARI          0.6972±0.00                  0.6685±0.04  0.6890±0.12
                                                   FMI          0.7645±0.00                  0.7421±0.04  0.7585±0.09
                                                   NMI          0.8028±0.00                  0.7925±0.01  0.7981±0.04
Heart Disease                                      ARI  

<br><br><br>
The results show that the Bernoulli Mixture clustering performs best across the board, followed closely by the Stochastic Bernoulli Mixture. KModes clustering seems to struggle with complex or large datasets.

The reason FMI scores tend to be higher than ARI and NMI is because it focuses on the overall number of correct assignments, regardless of how the data is actually grouped. This can make it seem better than it actually is, especially when dealing with messy clusters.

ARI and NMI, on the other hand, are stricter judges. They take into account how well the predicted clusters align with the true groupings and penalize both situations where clusters are split too much (over-segmentation) and where they are not distinct enough (under-segmentation). Additionally, ARI considers the possibility of random chance and adjusts the score accordingly, which can lead to lower values overall. NMI looks at the information shared between the clustering and the true labels, and if the clusters don't capture the natural groupings well, the score will suffer.